In [ ]:
# Installs

pip install gitpython beautifulsoup4 requests

In [ ]:
# Imports

import requests, time
from bs4 import BeautifulSoup
import os, json
from collections import defaultdict

In [ ]:
# Scrape topic from Github

topic_set = set()

for i in range(7):
    time.sleep(1)
    page = requests.get(f"https://github.com/topics?page={i}")
    # print(soup)
    soup = BeautifulSoup(page.content, 'html.parser')

    for find in soup.find_all('a'):
        topic = find['href']
        if topic.startswith("/topics/"):
            topic_set.add(topic)

    print(i, len(topic_set))

print("Have topic set")


In [ ]:
# Scrape repo names from topics - doesn't work
topic_set = set()
topic_set.add("/topics/cpp")

for topic in topic_set:
    print(topic)
    time.sleep(1)
    page = requests.get(f"https://github.com{topic}?o=desc&s=stars")
    soup = BeautifulSoup(page.content, 'html.parser')

    for a in soup.find_all('a', {"data-ga-click":"Explore, go to repository, location:explore feed"}):
        print(a['href'])
    break

In [ ]:
# Get repos

repos = [line.strip() for line in open('repos.txt')]

repo_contrib_dict = defaultdict(list)

for repo in repos:
    repo_name = '{}/{}'.format(*repo.split("/")[-2:])

    stream = os.popen(f'''
        rm -rf git-test > /dev/null;
        git clone {repo} git-test > /dev/null;
        cd git-test;
        git log --all --format=\'%aE\' | sort -u
    ''')
    contributor_emails = stream.read().split("\n")[:-1]

    for email in contributor_emails:
        repo_contrib_dict[email].append(repo_name)

    print(f"Finished {repo}")

with open('contributors2.json', 'w') as outfile:
    json.dump(repo_contrib_dict, outfile)

print("All done")